In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [3]:
df = pd.read_csv('Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
#Preprocess the data
df = df.drop(['RowNumber','CustomerId','Surname'],axis=1)
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [5]:
#Encode categorical variable
label_encoder_gender = LabelEncoder()
df['Gender'] = label_encoder_gender.fit_transform(df['Gender'])
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [6]:
#OneHot Encode Geographical Column
from sklearn.preprocessing import OneHotEncoder
onehot_encode_geo = OneHotEncoder()
geo_encoder = onehot_encode_geo.fit_transform(df[['Geography']])
geo_encoder

<10000x3 sparse matrix of type '<class 'numpy.float64'>'
	with 10000 stored elements in Compressed Sparse Row format>

In [7]:
geo_encoded_df = pd.DataFrame(geo_encoder.toarray(),columns=onehot_encode_geo.get_feature_names_out(['Geography']))
geo_encoded_df 

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [8]:
#Combine  onehot encoder column with original data
df = pd.concat([df.drop('Geography',axis=1),geo_encoded_df],axis=1)
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [9]:
#Split data into features and target
X = df.drop('EstimatedSalary',axis=1)
y = df['EstimatedSalary']

In [11]:
#Split the data in training and test set
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

#Scale these features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [13]:

#Save the encoders and scaler
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('onehot_encode_geo.pkl','wb') as file:
    pickle.dump(onehot_encode_geo,file) 
    
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

### ANN Regression

In [14]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [15]:
#Build ANN Model
model = Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)),  #Hidden Layer 1 connected with input layer 
    Dense(32,activation='relu'), #Hidden Layer 2
    Dense(1) #Output Layer for regression
])

2024-12-12 02:00:58.242354: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2 Pro
2024-12-12 02:00:58.242376: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-12-12 02:00:58.242383: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-12-12 02:00:58.242649: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-12-12 02:00:58.246459: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [17]:
#Compile the model
model.compile(optimizer='adam',loss='mean_absolute_error',metrics=['mae'])

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [20]:
#Set up the Tensorboard
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime
log_dir = "regressionlogs/fit" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callbacks = TensorBoard(log_dir=log_dir,histogram_freq=1)

In [21]:
#Setup Early Stopping
early_stopping_callbacks = EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [22]:
#Train the model
history = model.fit(
    X_train,y_train,validation_data=(X_test,y_test),epochs=100,
    callbacks=[tensorflow_callbacks,early_stopping_callbacks]
)

Epoch 1/100


2024-12-12 02:04:03.729617: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


250/250 [==============================] - 1s 5ms/step - loss: 100426.0938 - mae: 100426.0938 - val_loss: 98707.3672 - val_mae: 98707.3672
Epoch 2/100
250/250 [==============================] - 1s 4ms/step - loss: 100376.4531 - mae: 100376.4531 - val_loss: 98622.8516 - val_mae: 98622.8516
Epoch 3/100
250/250 [==============================] - 1s 3ms/step - loss: 100263.9219 - mae: 100263.9219 - val_loss: 98464.5156 - val_mae: 98464.5156
Epoch 4/100
250/250 [==============================] - 1s 4ms/step - loss: 100079.1953 - mae: 100079.1953 - val_loss: 98219.2500 - val_mae: 98219.2500
Epoch 5/100
250/250 [==============================] - 1s 4ms/step - loss: 99814.6484 - mae: 99814.6484 - val_loss: 97885.2266 - val_mae: 97885.2266
Epoch 6/100
250/250 [==============================] - 1s 4ms/step - loss: 99482.1328 - mae: 99482.1328 - val_loss: 97482.9375 - val_mae: 97482.9375
Epoch 7/100
250/250 [==============================] - 1s 4ms/step - loss: 99079.0312 - mae: 99079.0312 - val_

In [24]:
%load_ext tensorboard

In [27]:
%tensorboard --logdir regressionlogs/fit20241212-020344

Reusing TensorBoard on port 6006 (pid 3494), started 0:00:20 ago. (Use '!kill 3494' to kill it.)

In [28]:
#Evaluate the model
test_loss, test_mae = model.evaluate(X_test,y_test)
print(f"Test MAE: {test_mae}")

63/63 [==============================] - 0s 3ms/step - loss: 63500.0586 - mae: 63500.0586
Test MAE: 63500.05859375


In [29]:
model.save('model_regression.h5')

/Users/rishaan/miniforge3/envs/tensorflow-keras/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
